In [18]:
from bs4 import BeautifulSoup
import requests
import lxml
import pymysql

# Partie 1 : Chargement des données et création de la BD relationnelle sous MySQL

1). Analyser les données d’indicateurs du RGPH2014 selon les 6 thèmes (au niveau des
communes) puis définir le schéma global de la BD relationnelle qui va contenir ces
données.

<img src="Schéma global de la BD HCP.PNG">

**2).Créer une BD MySQL**

In [22]:
url= 'http://rgphentableaux.hcp.ma/Default1/'
headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36",
    "Referer":"http://rgphentableaux.hcp.ma/"
}

s = requests.Session()
s.headers.update(headers)
r = s.get(url)
soup = BeautifulSoup(r.content , "lxml")

In [23]:
selectCommune = soup.find('select', {'id':'COMMUNELIST'})

In [ ]:
connection = pymysql.connect(host='192.168.56.2', port=3306, user='hcp', password='Hcp-001', database='db_mysql_hcp')
cursor = connection.cursor()

In [ ]:
cursor.execute(
    """
        CREATE TABLE pays (
            id INT NOT NULL AUTO_INCREMENT,
            code VARCHAR ( 5 ) UNIQUE,
            nom VARCHAR ( 50 ) NOT NULL,
        PRIMARY KEY ( id ) 
        );
    """
)

In [ ]:
cursor.execute(
    """
        CREATE TABLE regions (
            id INT NOT NULL AUTO_INCREMENT,
            code VARCHAR ( 10 ) UNIQUE,
            nom VARCHAR ( 50 ) NOT NULL,
            code_pays VARCHAR ( 5 ),
            PRIMARY KEY ( id ),
        CONSTRAINT fk_pays FOREIGN KEY ( code_pays ) REFERENCES pays ( code ) 
        );
    """
)

In [ ]:
cursor.execute(
    """
        CREATE TABLE provinces (
            id INT NOT NULL AUTO_INCREMENT,
            code VARCHAR ( 20 ) UNIQUE,
            nom VARCHAR ( 50 ) NOT NULL,
            code_region VARCHAR ( 10 ),
            PRIMARY KEY ( id ),
        CONSTRAINT fk_region FOREIGN KEY ( code_region ) REFERENCES regions ( code ) 
        );
    """
)

In [ ]:
tb_communes = """
CREATE TABLE communes 
(id INT NOT NULL AUTO_INCREMENT,
code_commune VARCHAR(20) UNIQUE , 
code_province VARCHAR ( 20 ), 
nom_commune VARCHAR(50) NOT NULL,
PRIMARY KEY ( id ) , 
CONSTRAINT fk_province FOREIGN KEY ( code_province ) REFERENCES provinces ( code ) ) ;
"""
cursor.execute(tb_communes)

In [ ]:
cursor.execute(
    """
        CREATE TABLE themes (
            id INT NOT NULL AUTO_INCREMENT,
            code VARCHAR ( 10 ) UNIQUE,
            libelle VARCHAR ( 50 ) NOT NULL,
            PRIMARY KEY ( id )
        );
    """
)

In [ ]:
cursor.execute(
    """
        CREATE TABLE indicateurs (
            id INT NOT NULL AUTO_INCREMENT,
            code_theme VARCHAR ( 10 ),
            libelle VARCHAR ( 200 ) NOT NULL,
            PRIMARY KEY ( id ),
        CONSTRAINT fk_theme FOREIGN KEY ( code_theme ) REFERENCES themes ( code ) 
        );
    """
)

------------------------------------------------------------------------------------------------

**Creation des tables pour les Themes :**

------------------------------------------------------------------------------------------------

In [19]:
def supprime_accent( ligne):
        """ supprime les accents du texte source """
        print (ligne)
        accents = { 'a': ['à', 'ã', 'á', 'â'],
                    'e': ['é', 'è', 'ê', 'ë', 'É'],
                    'i': ['î', 'ï'],
                    'u': ['ù', 'ü', 'û'],
                    'o': ['ô', 'ö'],
                    '*': [' ','.','dans' ,'(',')',',', 'et*' ,'le*' , 'la*' ,'-','*a*' , 'les*', 'des*' , 'selon', 'l\'' , 'd\'' , 'de*' , 'du*'],
                    '_': ['*****', '****', '***', '**', '*'],
                    '': ['repartition_' , 'population_' , 'abetisee']
                  }
        for char, accented_chars in accents.items():
            for accented_char in accented_chars:
                ligne = ligne.replace(accented_char, char)
        return ligne

In [ ]:
list_themes = {
    '2':'demographie',
    '3':'handicap',
    '4':'education_alphabetisation',
    '5':'langues_locales',
    '6':'activite_emploi',
    '7':'conditions_habitat'
}

for k, v in list_themes.items():
    name_table = v
    valueB = 1
    length = 4
    if k == 7 :
        valueB = 4
        length = 3
    url = 'http://rgphentableaux.hcp.ma/Default1/getDATA/?type=Maroc&CGEO=&them='+k+''
    he ={'Referer': 'http://rgphentableaux.hcp.ma/Default1/'}
    dataOut = requests.get(url,headers = he ).json()     
    listColumn = []

    ##################################################################################""
    for line in dataOut :
        indicateur = line.get('INDICATEUR')
        dataB = line.get('DATA_B')
        #if dataB  == 1 :
        if dataB  == valueB :
            #indicateur = indicateur[4:].lower()
            indicateur = indicateur[length:].lower()
            indicateur = supprime_accent(indicateur)
            if indicateur not in listColumn:
                listColumn.append(indicateur)
    ############################################################################""
    query = "CREATE TABLE "+name_table+" ( id INT NOT NULL AUTO_INCREMENT, code_commune VARCHAR (20)"
    for col in listColumn:
        query = query + ", "+col+" DOUBLE"
    query = query + ", PRIMARY KEY ( id ), CONSTRAINT fk_"+name_table + "_commune FOREIGN KEY ( code_commune ) REFERENCES communes ( code_commune ));"

    cursor.execute(query)

**3) Charger les données dans cette BD :**

In [ ]:
#cherger les données dans la tables Regions
selectRegion = soup.find('select', {'id':'REGIONSLIST'})

regions = ""
for option in selectRegion.find_all('option'):
    regions = regions + "(\"" + option['value'].strip() + "\" , \""+ option.text.strip() + "\" , \"1\"),"
regions = regions.strip()[:-1] + ";"

insert_regions = "insert into regions (code , nom , code_pays ) values " + regions

cursor.execute(insert_regions)
connection.commit()

In [ ]:
#cherger les données dans la tables Provinces
selectProvince = soup.find('select', {'id':'PROVINCESLIST'})

provinces = ""
for option in selectProvince.find_all('option'):
    provinces = provinces + "(\"" + option['value'].strip() + "\" , \""+ option.text.strip() + "\" , \"01\"),"
provinces = provinces.strip()[:-1] + ";"

insert_provinces = "insert into provinces (code , nom , code_region ) values " + provinces

cursor.execute(insert_provinces)
connection.commit()

In [ ]:
#cherger les données dans la tables communes
selectCommune = soup.find('select', {'id':'COMMUNELIST'})

communes = ""
for option in selectCommune.find_all('option'):
    #print('value: '+option['value'].strip()[:-1] +', text: '+option.text)
    communes = communes + "(\"" + option['value'].strip()[:-1] + "\" , \""+ option.text.strip() + "\" , \""+option['value'].strip()[:3]+"\"),"
communes = communes.strip()[:-1] + ";"

insert_communes = "insert into communes (code_commune , nom_commune , code_province ) values " + communes
cursor.execute(insert_communes)
connection.commit()

In [ ]:
#Ajouter les clés étrangères(regions) dans la table provinces
import json

with open('regions_provinces.json') as json_file:
    data = json.load(json_file)
    
for attribute , value in data.items():  
    query = "update provinces set code_region = '"+value['code_region']+"' where code ='"+attribute+"';"
    #print(query)
    cursor.execute(query)
    connection.commit()

In [27]:
#la fonction getData permet de charger les données dans la table donner en parametres
def getData( code_theme , table , code_commune ):
    url = "http://rgphentableaux.hcp.ma/Default1/getDATA/?type=Commune&CGEO="+code_commune+".&them="+code_theme
    header ={'Referer': 'http://rgphentableaux.hcp.ma/Default1/'}
    dataOut = requests.get(url,headers = header ).json() 

    insert_queryUF = "insert into "+ table +" values (NULL,'"+ code_commune +"',"
    insert_queryUM = "insert into "+ table +" values (NULL,'"+ code_commune +"',"
    insert_queryRF = "insert into "+ table +" values (NULL,'"+ code_commune +"',"
    insert_queryRM = "insert into "+ table +" values (NULL,'"+ code_commune +"',"
    
    for line in dataOut :
        indicateur = line.get('INDICATEUR')
        zone_geographique = line.get('DATA_B')
    
        valeur = line.get('DATA2014').replace(',','.')
        try:
            observation = float(valeur)
        except ValueError:
            observation = 0
        
        if zone_geographique == 2 :
            if indicateur[2:3] == 'M':
                insert_queryUM = insert_queryUM + str(observation)+","
            if indicateur[2:3] == 'F':    
                insert_queryUF = insert_queryUF + str(observation)+","
            if indicateur[2:3] == '_':
                insert_queryUM = insert_queryUM + str(observation)+","
                
        if zone_geographique == 3 :
            if indicateur[2:3] == 'M':
                insert_queryRM = insert_queryRM + str(observation)+","
            if indicateur[2:3] == 'F':
                insert_queryRF = insert_queryRF + str(observation)+","
            if indicateur[2:3] == '_': 
                insert_queryRM = insert_queryRM + str(observation)+","
    
    if code_theme == '7' :
        insert_queryUM = insert_queryUM  +"2);"
        insert_queryRM = insert_queryRM  +"3);"
        cursor.execute(insert_queryUM)
        cursor.execute(insert_queryRM)
    else : 
        insert_queryUM = insert_queryUM  +"0,2);"            
        insert_queryUF = insert_queryUF  +"1,2);"
        insert_queryRM = insert_queryRM  +"0,3);"            
        insert_queryRF = insert_queryRF  +"1,3);"
        print(insert_queryUM)
        print(insert_queryUF)
        print(insert_queryRM)
        print(insert_queryRF)
        cursor.execute(insert_queryUM)
        cursor.execute(insert_queryUF)
        cursor.execute(insert_queryRM)
        cursor.execute(insert_queryRF)
    connection.commit()

In [ ]:
#Liste des communes
list_communes = []
for option in selectCommune.find_all('option'):
    list_communes.append(option['value'].strip()[:-1]) 
list_communes

#liste des themes
list_themes = {
    '2':'demographie',
    '3':'handicap',
    '4':'education_alphabetisation',
    '5':'langues_locales',
    '6':'activite_emploi',
    '7':'conditions_habitat'
}

#Charger les données dans cette BD
for code, theme in list_themes.items():
    for codeCommune in list_communes:
        getData( code , theme , codeCommune )

# Partie 2 : Interrogation de la BD MySQL

In [51]:
#Quel est le total de la population municipale du Maroc

queryQ4 ="""SELECT sum(population_municipale) from 
demographie , communes , provinces , regions , pays
where 
demographie.code_commune = communes.code_commune
and communes.code_province = provinces.code
and provinces.code_region = regions.code
and regions.code_pays = pays.code
and pays.code = '1';"""
nb_rows = cursor.execute(queryQ4)
dataOutQ4 = cursor.fetchall()
print(dataOutQ4)

((33610084.0,),)


In [52]:
#Quels sont les dix premières communes les plus peuplées du Maroc

queryQ5 ="""select sum(demographie.population_municipale) as sum_population_municipale , communes.nom_commune 
from demographie , communes
where demographie.code_commune = communes.code_commune
group by demographie.code_commune , communes.nom_commune
order by sum_population_municipale desc
limit 10;"""
nb_rows = cursor.execute(queryQ5)
dataOutQ5 = cursor.fetchall()
dataOutQ5

((517376.0, 'Meknès'),
 (492873.0, 'Oujda'),
 (467880.0, 'Hay-Hassani'),
 (452863.0, 'Sidi Moumen'),
 (423890.0, 'Kénitra'),
 (420288.0, 'Agadir'),
 (409829.0, 'Ménara'),
 (385922.0, 'Bni Makada'),
 (377866.0, 'Tétouan'),
 (376772.0, 'Aîn-Chock'))

In [53]:
#Quel est le nombre de communes existant hors les régions de Rabat et Casablanca ayant une
#population municipale supérieur à 50 000 habitants, un taux de chômage inférieur à 15%, un
#Taux net d'activité féminin supérieur à 30% et un taux d’analphabétisme inférieur à 20%

queryQ6 ="""select count(communes.code_commune) as nb_communes 
from communes , provinces , regions  
where communes.code_province = provinces.code
and provinces.code_region = regions.code
and regions.code not in ('04','06')
and communes.code_commune in 
    (select actEmp.code_commune from activite_emploi as actEmp 
    group by actEmp.code_commune
    having sum(actEmp.taux_chomage)/2 > 15 
    and sum(actEmp.taux_n_activite)/2 > 30 )
and communes.code_commune in 
    (select edAlph.code_commune from education_alphabetisation edAlph
    group by edAlph.code_commune 
    having sum(edAlph.taux_analphabetisme)/2 > 20 )
and communes.code_commune in 
    (select demo.code_commune from demographie as demo 
    group by demo.code_commune 
    having  sum(demo.population_municipale)>50000);"""
nb_rows = cursor.execute(queryQ6)
dataOutQ6 = cursor.fetchall()
dataOutQ6

((51,),)

In [54]:
#Quelle est la province ayant le taux de prévalence d’handicap le plus faible

queryQ7 ="""select provinces.nom , avg(han_s.taux_prevalence) as Min_Taux_Prevalence
from handicap ha 
join (select han.code_commune as code_commune , sum(han.taux_prevalence_handicap)/2 as taux_prevalence
    from handicap han 
    group by han.code_commune) as han_s
on han_s.code_commune = ha.code_commune
join communes 
on communes.code_commune = ha.code_commune
join provinces 
on provinces.code = communes.code_province
group by communes.code_province , provinces.nom
order by Min_Taux_Prevalence asc
limit 1;"""
nb_rows = cursor.execute(queryQ7)
dataOutQ7 = cursor.fetchall()
dataOutQ7

(('Oued Ed-Dahab', 0.9),)

In [55]:
#Quel est l’âge moyen au premier mariage par genre (masculin, féminin) et par secteur
#(urbain, rural) au niveau de chaque région ?

queryQ8 ="""select regions.nom , avg(demo.age_moyen_au_premier_mariage)  , 
case when demo.sexe = 0 then 'Masculin'
else 'Féminin' end as sexe
from demographie as demo
join communes
on communes.code_commune = demo.code_commune
join provinces
on provinces.code = communes.code_province
join regions
on provinces.code_region = regions.code
group by demo.sexe , categorie_territoriale , regions.code , regions.nom;"""
nb_rows = cursor.execute(queryQ8)
dataOutQ8 = cursor.fetchall()
dataOutQ8

(('Tanger-Tetouan-Al Hoceima', 32.229729729729726, 'Masculin'),
 ('Oriental', 32.962790697674414, 'Masculin'),
 ('Fès-Meknès', 31.844615384615395, 'Masculin'),
 ('Rabat-Salé-Kénitra', 31.643478260869575, 'Masculin'),
 ('Béni Mellal-Khénifra', 31.59736842105264, 'Masculin'),
 ('Casablanca-Settat', 31.518965517241373, 'Masculin'),
 ('Marrakech-Safi', 30.885106382978726, 'Masculin'),
 ('Drâa-Tafilalet', 30.726923076923075, 'Masculin'),
 ('Souss-Massa', 30.82857142857143, 'Masculin'),
 ('Guelmim-Oued Noun', 32.1, 'Masculin'),
 ('Laayoune-Sakia El Hamra', 31.32, 'Masculin'),
 ('Eddakhla-Oued Eddahab', 15.5, 'Masculin'),
 ('Tanger-Tetouan-Al Hoceima', 31.37286821705425, 'Masculin'),
 ('Oriental', 31.948958333333334, 'Masculin'),
 ('Fès-Meknès', 30.648447204968964, 'Masculin'),
 ('Rabat-Salé-Kénitra', 30.17692307692308, 'Masculin'),
 ('Béni Mellal-Khénifra', 29.748739495798326, 'Masculin'),
 ('Casablanca-Settat', 30.228225806451604, 'Masculin'),
 ('Marrakech-Safi', 29.347807017543836, 'Mascul

In [56]:
#Dans quelles provinces se trouvent-elles les dix communes ayant le taux d’analphabétisme
#le plus élevé
queryQ9 ="""
select distinct provinces.nom 
from provinces
join communes
on provinces.code = communes.code_province
join (
    select education_alphabetisation.code_commune as code_communes , sum(taux_analphabetisme)/2 as analphabetisme
    from education_alphabetisation
    group by education_alphabetisation.code_commune  
    order by analphabetisme desc
    limit 10
    ) top_communes
on top_communes.code_communes = communes.code_commune;"""
nb_rows = cursor.execute(queryQ9)
dataOutQ9 = cursor.fetchall()
dataOutQ9

(('Figuig',),
 ('Taourirt',),
 ('Boujdour',),
 ('Midelt',),
 ('Jerada',),
 ('Taroudannt',),
 ('Tétouan',))

In [58]:
#Quelles sont les provinces où la langue locale « Tachelhit » est plus utilisée que
#« Tamazight » dans le cas où le pourcentage cumulé de ces deux langues locales dépasse
#50% ?
queryQ11 ="""
select distinct provinces.nom 
from provinces 
join communes on provinces.code = communes.code_province
join(
select langues_locales.code_commune as code_commune , 
sum(langues_loca_utilisees_non_exclusives__tachelhit)/2 as tachelhit , 
sum(langues_loca_utilisees_non_exclusives__tamazight)/2 as tamazight 
from langues_locales
group by langues_locales.code_commune
having tachelhit > tamazight 
and (tachelhit + tamazight )  > 50) list_provinces
on list_provinces.code_commune = communes.code_commune;"""
nb_rows = cursor.execute(queryQ11)
dataOutQ11 = cursor.fetchall()
dataOutQ11

(('Agadir Ida Ou Tanane',),
 ('Al Haouz',),
 ('Al Hoceima',),
 ('Azilal',),
 ('Chichaoua',),
 ('Chtouka Ait Baha',),
 ('Essaouira',),
 ('Guelmim',),
 ('Inezgane Ait Melloul',),
 ('Ouarzazate',),
 ('Sidi Ifni',),
 ('Taroudannt',),
 ('Tata',),
 ('Tiznit',),
 ('Zagora',))

In [59]:
#Quel est le nombre de provinces où le taux d’équipement des ménages en parabole est
#supérieur à celui en réfrigérateur ?

queryQ12 ="""
select count(distinct code_province) as nbProvinces 
from communes
join(
select conditions_habitat.code_commune as code_commune , 
sum(conditions_habitat.autres_equipements_menagers_parabole)/2 as parabole,
sum(conditions_habitat.autres_equipements_menagers_refrigerateur)/2 as refrigerateur
from conditions_habitat
group by conditions_habitat.code_commune
having  parabole > refrigerateur
) cndHabitat
on cndHabitat.code_commune = communes.code_commune ; """
nb_rows = cursor.execute(queryQ12)
dataOutQ12 = cursor.fetchall()
dataOutQ12

((65,),)